In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 60)
plt.rcParams['figure.figsize'] = (10,6)


In [3]:
sample_size = 500000  
df = pd.read_csv('../data/raw/US_Accidents.csv', 
                 low_memory=False, 
                 nrows=sample_size)

In [4]:
cols_to_drop = ['End_Lng', 'End_Lat', 'Precipitation(in)', 'Wind_Chill(F)']
cols_to_drop


['End_Lng', 'End_Lat', 'Precipitation(in)', 'Wind_Chill(F)']

In [6]:
num_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
cat_cols = df.select_dtypes(include=['object']).columns.tolist()

num_medians = df[num_cols].median(numeric_only=True).to_dict()

cat_modes = {}
for c in cat_cols:
    try:
        mode_val = df[c].mode(dropna=True)
        cat_modes[c] = mode_val.iloc[0] if len(mode_val) > 0 else "Unknown"
    except:
        cat_modes[c] = "Unknown"

num_medians, list(cat_modes.items())[:10]


({'Severity': 2.0,
  'Start_Lat': 34.2472495,
  'Start_Lng': -95.543968,
  'End_Lat': nan,
  'End_Lng': nan,
  'Distance(mi)': 0.01,
  'Temperature(F)': 66.2,
  'Wind_Chill(F)': 31.0,
  'Humidity(%)': 66.0,
  'Pressure(in)': 30.0,
  'Visibility(mi)': 10.0,
  'Wind_Speed(mph)': 8.1,
  'Precipitation(in)': 0.01},
 [('ID', 'A-1'),
  ('Source', 'Source2'),
  ('Start_Time', '2016-04-10 08:59:26'),
  ('End_Time', '2016-10-14 19:50:00'),
  ('Description', 'Accident on I-80 Westbound at Exits 8A 8B I-880.'),
  ('Street', 'I-5 N'),
  ('City', 'Houston'),
  ('County', 'Los Angeles'),
  ('State', 'CA'),
  ('Zipcode', '91706')])

In [12]:
def clean_df(df_in):
    dfc = df_in.copy()

    if 'Start_Time' in dfc.columns:
        dfc['Start_Time'] = pd.to_datetime(dfc['Start_Time'], errors='coerce')
    if 'End_Time' in dfc.columns:
        dfc['End_Time'] = pd.to_datetime(dfc['End_Time'], errors='coerce')

    if 'Start_Time' in dfc.columns:
        dfc['Hour'] = dfc['Start_Time'].dt.hour
        dfc['Weekday'] = dfc['Start_Time'].dt.day_name()
        dfc['Month'] = dfc['Start_Time'].dt.month

    dfc.drop(columns=cols_to_drop, inplace=True, errors='ignore')

    for c, med in num_medians.items():
        if c in dfc.columns:
            dfc[c] = dfc[c].fillna(med)

    for c, mode in cat_modes.items():
        if c in dfc.columns:
            dfc[c] = dfc[c].fillna(mode)

    if 'Visibility(mi)' in dfc.columns:
        dfc['Visibility(mi)'] = dfc['Visibility(mi)'].clip(0, 50)

    before = dfc.shape[0]
    dfc.drop_duplicates(inplace=True)
    after = dfc.shape[0]

    print(f"Removed duplicates: {before - after}")

    return dfc


In [13]:
cleaned_df = clean_df(df)
cleaned_df.shape


Removed duplicates: 0


(500000, 45)

In [11]:
cleaned_df.to_csv('../data/cleaned/US_Accidents_cleaned_sample_milestone1.csv', index=False)
print("Cleaned sample saved successfully!")


Cleaned sample saved successfully!
